In [1]:
using Oscar


In [2]:
function text_reader(name)
    f = open(name)
    lines = readlines(f)
    E = []
    v= 0
    for l in lines
        x = l[1]
        if x == 'p'
            t = split(l)
            v = parse(Int64, t[3])
        elseif x == 'e'
            t = split(l)
            push!(E, [parse(Int64,t[2]), parse(Int64,t[3])]) 
        end
    end
return v,E
end

function polisGrafo(g,k,R)
    # Devuelve dado un grafo y un entero k, un sistema de polinomios que ayuda a determinar que el grafo es k-colorable Y el anillo de polinomios
    ps = [R[j]^k-1 for j in 1:nv(g)] # los polinomios x^k_i- 1 = 0, ∀i ∈ V(g)
    ps2 = [R[src(r)]^(k-1-l) * R[dst(r)]^(l) for r in collect(edges(g)), l in 0:k-1] # los polinomios sum_l=0^{k-1} x_i^{k-1-l} x_j^l=0 ∀{i, j} ∈ E(G)
    p2 = sum(ps2,dims=2)
    polis = vec(vcat(ps,p2)) # crea una lista con los polinomios
    return polis # retorna la lista de polinomios 
end

polisGrafo (generic function with 1 method)

In [3]:
nver, E = text_reader("Grafos/johnson8-2-4.clq.txt")
g = Graph{Undirected}(nver)
for v in E
    add_edge!(g, v[1], v[2])
end

In [4]:
nve, E = 4, [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
g = Graph{Undirected}(nve)
for v in E
    add_edge!(g, v[1], v[2])
end


In [5]:
#  Calcula las particiones de d en sumas de n numeros, pueden ser 0
function partition(n::Int, d::Int)
    if n == 1
        return [[d]]
    end
    partitions = []
    for i in 0:d
        for p in partition(n-1, d-i)
            push!(partitions, [i; p])
        end
    end
    return partitions
end
# Calcula los polinomios beta_i sin coeficientes
function polysBi(n::Int, d::Int,genes)
    v = []
    
    for j in 0:d
        v = vcat(v, [reduce(*,[genes[i]^par[i] for i in 1:length(par)]) for par in partition(n,j)])
    end
    return v
end
# Retorna los polinomios beta_i * f_i sin los coeficientes
function creaCertv(d::Int,listap)
    v=[]
    n=length(gens(parent(listap[1])))
    for f in listap
        dd = total_degree(f)
        v = vcat(v,[ f * bi for bi in polysBi(n, d - dd,gens(parent(listap[1])))])
    end
    return v
end

# Calcula el numero de coefficientes
ncoefi(n::Int,e::Int,d::Int,k::Int) = sum(fill(binomial(n+d-k,d-k), n)) + sum(fill(binomial(n + d -k +1,d-k+1), e))::Int

# Pasar a entero
aEntero(a) = (a == GF(2)(1)) ? 1 : (a == GF(2)(0)) ? 0 : nothing

aEntero (generic function with 1 method)

In [6]:
function nulla(g::Graph,cota::Int,k::Int)::Vector{Float64}
    n = nv(g)
    e = ne(g)
    d = 3
    while d <= cota
        nUnkows = ncoefi(n,e,d,k)
        GF2 = GF(2)
        T, y =  PolynomialRing(GF2,["y$i" for i in 1:nUnkows])
        R2, z = PolynomialRing(T,["z$i" for i in 1:n])

        PG = polisGrafo(g, k, R2)

        CERTsinC2 = creaCertv(d,PG)
        CERTv = [ y[i]*CERTsinC2[i] for i in  1:length(CERTsinC2)]

        CERT = reduce(vcat,[collect(terms(f)) for f in CERTv])

        CERT1 = sum(CERT)
        CERTt = collect(terms(CERT1))
        println(CERTt)
        # Creacion del sistema lineal 
        A = []
        for cee in CERTt
            achiquita = vec([aEntero(coeff(cc, yi)) for yi in gens(T), cc in collect(coefficients(cee))])
            if isempty(A)
                A = achiquita
            else
                A=hcat(A,achiquita)
            end
        end
        A = transpose( A)
        println(A)
        #b = fill(GF(2)(0),length(CERTt))
        #b[end]+=GF2(1)
        b = zeros(Int,length(CERTt))
        println(length(CERTt))
        b[end]+=1
        sol = A\b
        if any(sol .!= 0) # si no es todo 0, el sistema es consistente
            println("The system of equations F is infeasible")
            return sol
        end
        println("Una iteracion")
        d += 1
    end
    println("The system of equations F is feasible.")
    return true
end

nulla (generic function with 1 method)

In [8]:
nulla(g,56,3)

AbstractAlgebra.Generic.MPoly{fpMPolyRingElem}[(y1 + y9 + y14 + y24)*z1^3, (y8 + y9 + y13 + y23)*z1^2*z2, (y8 + y9 + y19 + y29)*z1*z2^2, (y2 + y8 + y18 + y28)*z2^3, (y7 + y12 + y14 + y22)*z1^2*z3, (y7 + y13 + y19)*z1*z2*z3, (y7 + y17 + y18 + y27)*z2^2*z3, (y12 + y14 + y19 + y34)*z1*z3^2, (y13 + y17 + y18 + y33)*z2*z3^2, (y3 + y12 + y17 + y32)*z3^3, (y6 + y11 + y21 + y24)*z1^2*z4, (y6 + y23 + y29)*z1*z2*z4, (y6 + y16 + y26 + y28)*z2^2*z4, (y11 + y22 + y34)*z1*z3*z4, (y16 + y27 + y33)*z2*z3*z4, (y11 + y16 + y31 + y32)*z3^2*z4, (y21 + y24 + y29 + y34)*z1*z4^2, (y23 + y26 + y28 + y33)*z2*z4^2, (y22 + y27 + y31 + y32)*z3*z4^2, (y4 + y21 + y26 + y31)*z4^3, (y5 + y10 + y20)*z1^2, y5*z1*z2, (y5 + y15 + y25)*z2^2, y10*z1*z3, y15*z2*z3, (y10 + y15 + y30)*z3^2, y20*z1*z4, y25*z2*z4, y30*z3*z4, (y20 + y25 + y30)*z4^2, y1 + y2 + y3 + y4]
[1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0

; 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0; 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0; 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0; 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0; 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0; 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0; 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0; 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

34-element Vector{Float64}:
  0.22916666666666657
  0.2291666666666664
  0.2291666666666666
  0.22916666666666657
  1.393380451513459e-17
  0.027777777777777728
  0.027777777777777832
 -0.04861111111111104
 -0.04861111111111108
 -4.500139929349619e-18
  ⋮
 -0.04861111111111112
  0.027777777777777707
 -0.048611111111111105
  0.027777777777777873
  1.3933804515134587e-17
 -0.04861111111111107
 -0.04861111111111108
  0.02777777777777777
  0.027777777777777762